In [1]:
from pyocd.core.helpers import ConnectHelper
from pyocd.flash.file_programmer import FileProgrammer
from pyocd.debug.elf.symbols import ELFSymbolProvider
import inspect

import logging
logging.basicConfig(level=logging.INFO)

session = ConnectHelper.session_with_chosen_probe()
board = session.board
target = board.target
flash = target.memory_map.get_boot_memory()
mmap= target.memory_map
session.open()

INFO:pyocd.board.board:Target type is stm32f446retx
INFO:pyocd.coresight.dap:DP IDR = 0x2ba01477 (v1 rev2)
INFO:pyocd.coresight.ap:AHB-AP#0 IDR = 0x24770011 (AHB-AP var1 rev2)
INFO:pyocd.coresight.rom_table:AHB-AP#0 Class 0x1 ROM table #0 @ 0xe00ff000 (designer=020:ST part=411)
INFO:pyocd.coresight.rom_table:[0]<e000e000:SCS v7-M class=14 designer=43b:Arm part=00c>
INFO:pyocd.coresight.rom_table:[1]<e0001000:DWT v7-M class=14 designer=43b:Arm part=002>
INFO:pyocd.coresight.rom_table:[2]<e0002000:FPB v7-M class=14 designer=43b:Arm part=003>
INFO:pyocd.coresight.rom_table:[3]<e0000000:ITM v7-M class=14 designer=43b:Arm part=001>
INFO:pyocd.coresight.rom_table:[4]<e0040000:TPIU M4 class=9 designer=43b:Arm part=9a1 devtype=11 archid=0000 devid=ca1:0:0>
INFO:pyocd.coresight.rom_table:[5]<e0041000:ETM M4 class=9 designer=43b:Arm part=925 devtype=13 archid=0000 devid=0:0:0>
INFO:pyocd.coresight.cortex_m:CPU core #0 is Cortex-M4 r0p1
INFO:pyocd.coresight.cortex_m:FPU present: FPv4-SP-D16-M
INF

In [16]:
a=target.svd_device
a = a.peripherals[0]


In [15]:
session.user_script_proxy.namespace.keys()


dict_keys(['__builtins__', 'pyocd', 'exceptions', 'Error', 'TransferError', 'TransferFaultError', 'Target', 'State', 'SecurityState', 'BreakpointType', 'WatchpointType', 'VectorCatch', 'Event', 'RunType', 'HaltReason', 'ResetType', 'MemoryLoader', 'MemoryType', 'MemoryMap', 'RamRegion', 'RomRegion', 'FlashRegion', 'DeviceRegion', 'FileProgrammer', 'FlashEraser', 'FlashLoader', '__name__', '__file__', 'session', 'options', 'LOG', 'command', 'debug', 'info', 'warning', 'error', 'probe', 'board', 'target', 'dp', 'aps'])

In [18]:
a.registers.index

{'name': 'DCMI',
 'version': None,
 'derived_from': None,
 'description': 'Digital camera interface',
 'prepend_to_name': None,
 'base_address': 1342504960,
 'address_block': {'offset': 0, 'size': 1024, 'usage': 'registers'},
 'interrupts': [{'name': 'DCMI',
   'value': 78,
   'description': 'DCMI global interrupt'}],
 'registers': [{'derived_from': None,
   'name': 'CR',
   'description': 'control register 1',
   'address_offset': 0,
   'read_action': None,
   'modified_write_values': None,
   'display_name': 'CR',
   'alternate_group': None,
   'size': 32,
   'access': None,
   'protection': None,
   'reset_value': 0,
   'reset_mask': 4294967295,
   'fields': [{'name': 'CAPTURE',
     'derived_from': None,
     'description': 'Capture enable',
     'bit_offset': 0,
     'bit_width': 1,
     'access': 'read-only',
     'enumerated_values': None,
     'modified_write_values': None,
     'read_action': None},
    {'name': 'CM',
     'derived_from': None,
     'description': 'Capture mod

In [22]:
target.reset_and_halt()
FileProgrammer(session).program("F:\\Projects\\MicroController\\STM32F446RE\\sotom_v1\\build\\final.elf")
target.reset_and_halt()
target.resume()
target.reset()

[---|---|---|---|---|---|---|---|---|----]
[========================================]


INFO:pyocd.flash.loader:Erased 16384 bytes (1 sector), programmed 5120 bytes (5 pages), skipped 0 bytes (0 pages) at 4.93 kB/s


In [7]:
inspect.getsource(target.read_core_register)
#session.close()
session.board.unique_id
board.name
board.target_type
target.get_memory_map()[3]

target.elf = "F:\\Projects\\MicroController\\STM32F446RE\\sotom_v1\\build\\final.elf"
provider = ELFSymbolProvider(target.elf)
addr = provider.get_symbol_value("DEBUG_BUTTON")

print('addr:'+hex(addr))
sp = target.read16(addr)
print("val: "+bin(sp))


addr:0x2000000c
val: 0b100101


In [3]:
FlashTotal=0
SRamTotal=0
SRamEnd = 0
for m in target.get_memory_map():
    if str(m.type) == 'MemoryType.FLASH':
        FlashTotal += m.length
    if str(m.type) == 'MemoryType.RAM':
        SRamTotal += m.length
        SRamEnd = m.end

print(str(FlashTotal)+ " : " +str(SRamTotal)+ ' ' +str(SRamEnd))

524288 : 131072 537001983


In [10]:

a = target.read32(0x00000010)
print(hex(a))

for i in range(201):
    a = target.read32(0x20020000-0x4-0x4*i)
    print(str(hex(0x20020000-0x4-0x4*i))+'__:'+str(a))

a = target.read32(0x0)
print("Stack Pointer: "+hex(a))

0x8000b01
0x2001fffc__:4294967295
0x2001fff8__:0
0x2001fff4__:536870952
0x2001fff0__:134220980
0x2001ffec__:4
0x2001ffe8__:36
0x2001ffe4__:36
0x2001ffe0__:2510418743
0x2001ffdc__:134220693
0x2001ffd8__:537001952
0x2001ffd4__:6612370
0x2001ffd0__:56
0x2001ffcc__:56
0x2001ffc8__:56
0x2001ffc4__:56
0x2001ffc0__:56
0x2001ffbc__:56
0x2001ffb8__:56
0x2001ffb4__:56
0x2001ffb0__:56
0x2001ffac__:56
0x2001ffa8__:56
0x2001ffa4__:56
0x2001ffa0__:56
0x2001ff9c__:56
0x2001ff98__:56
0x2001ff94__:56
0x2001ff90__:56
0x2001ff8c__:56
0x2001ff88__:56
0x2001ff84__:56
0x2001ff80__:56
0x2001ff7c__:56
0x2001ff78__:56
0x2001ff74__:56
0x2001ff70__:56
0x2001ff6c__:56
0x2001ff68__:56
0x2001ff64__:56
0x2001ff60__:56
0x2001ff5c__:56
0x2001ff58__:56
0x2001ff54__:56
0x2001ff50__:56
0x2001ff4c__:56
0x2001ff48__:56
0x2001ff44__:56
0x2001ff40__:56
0x2001ff3c__:56
0x2001ff38__:56
0x2001ff34__:56
0x2001ff30__:56
0x2001ff2c__:56
0x2001ff28__:56
0x2001ff24__:56
0x2001ff20__:56
0x2001ff1c__:56
0x2001ff18__:56
0x2001ff14__:56

In [97]:
target.halt()

for i in range(32):
    hh = target.read_core_register_raw(i)
    print('R'+str(i)+' : '+hex(hh))

R0 : 0x4b0
R1 : 0xe000e100
R2 : 0x8000c60
R3 : 0x0
R4 : 0x0
R5 : 0x0
R6 : 0x0
R7 : 0x2001fe40
R8 : 0x0
R9 : 0x0
R10 : 0x0
R11 : 0x0
R12 : 0x0
R13 : 0x2001fe40
R14 : 0x80007e7
R15 : 0x80009a0
R16 : 0x1000000
R17 : 0x2001fe40
R18 : 0x0


KeyError: 'unknown core register 19'

In [5]:
target.halt()
o = 0
hh = target.read_core_register('msp')
hex(hh)

'0x2001fe40'

In [6]:
target.halt()
a = target.get_state()

a.name
target.rea

'HALTED'